# Lab 06 Softmax Classification
* Softmax를 사용하여분류를 진행합니다.

### 기본 Library 선언 및 Tensorflow 버전 확인

In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)  # for reproducibility
tfe = tf.contrib.eager

/Users/sinseongjin/tf110/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [3]:
#dataset을 선언합니다.
# dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
# dataset = dataset.repeat().batch(2)

nb_classes = 3 #class의 개수입니다.

In [4]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tfe.Variable(tf.random_normal([4, nb_classes]), name='weight')
        self.b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def loss_fn(self, X, Y):
        logits = self.softmax_regression(X)
        loss = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(logits), axis=1))
        
        return loss
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            loss = self.loss_fn(x_data, y_data)
            grads = tape.gradient(loss, self.variables)
            
            return grads
    
    def fit(self, X, Y, epochs=500, verbose=50):
        optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)
        
        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.loss_fn(X, Y).numpy()))

In [5]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
model = softmax_classifer(nb_classes)

In [6]:
model.fit(x_data, y_data)

Loss at epoch 1: 2.849417
Loss at epoch 50: 0.741152
Loss at epoch 100: 0.684151
Loss at epoch 150: 0.645958
Loss at epoch 200: 0.613813
Loss at epoch 250: 0.584977
Loss at epoch 300: 0.558204
Loss at epoch 350: 0.532792
Loss at epoch 400: 0.508306
Loss at epoch 450: 0.484460
Loss at epoch 500: 0.461059
